In [1]:
import json

## Function to find g, anharmonicity, qubit frequency

In [2]:
import numpy as np
import scqubits as scq

# Constants
e = 1.602e-19  # elementary charge in C
hbar = 1.054e-34  # reduced Planck constant in Js
Z_0 = 50  # in Ohms

def find_g_a(C_g, C_B, f_r, Lj, N):
    C_Sigma = C_g + C_B # + 1.5e-15
    omega_r = 2 * np.pi * f_r
    EJ = ((hbar / 2 / e) ** 2) / Lj * (1.5092e24) # 1J = 1.5092e24 GHz
    EC = e**2/(2*C_Sigma) * (1.5092e24) # 1J = 1.5092e24 GHz

    transmon = scq.Transmon(EJ=EJ,
                            EC=EC,
                            ng = 0,
                            ncut = 30)

    a = transmon.anharmonicity() * 1000 # linear MHz
    g = ((C_g / C_Sigma) * omega_r * np.sqrt(N * Z_0 * e**2 / (hbar * np.pi) )* (EJ/(8*EC))**(1/4)) / 1E6 / (2 * np.pi) # linear MHz
    f_q = transmon.E01() # Linear GHz
    
    return g, a, f_q

In [3]:
# predicted_cross_length = [211.36686180080213, 239.67001841727927, 199.29546197022296, 272.80405439803087, 166.21118594572997]
# predicted_claw_length = [62.74561331858793, 238.54614619784087, 166.65135380618355, 49.04728878924247, 122.98283288276569]
# Lj_vals = [10.130278903289723, 8.837126261668763, 10.545628251119771, 7.934490145874526, 12.130125654474945]

vec1 = (12.101812915800638, 165.60709433301977, 142.67234304675804)
vec2 = (10.092518176996967, 210.20946233643153, 201.9365897494747)
Lj_vals = []
predicted_cross_length = []
predicted_claw_length = []

for Lj, xlen, clen in [vec1, vec2]:
    Lj_vals += [Lj]
    predicted_cross_length += [xlen]
    predicted_claw_length += [clen]

In [4]:
index = 1

### Sanity check with a known qubit (to make sure my code still works)

In [5]:
# Qubit Identifier 1

C_g = 5.91536e-15	# in F
C_B = 122.44002e-15	# in F
f_r = 6.116e9 # linear frequency in Hz
f_q = 4.216 # Linear freq in GHz
Lj = 10e-9
anharmonicity = -0.153 # Linear freq in GHz


print('Qubit 1')
print("Experimental g = 61 Linear MHz")
# print(f"Simulated g = {g(C_g, C_B, f_r, f_q, anharmonicity, 4) / 1E6 / (2 * np.pi)} Linear MHz")
print(f"Simulated g = {find_g_a(C_g, C_B, f_r, Lj, 4)[0]} Linear MHz")
print(f"Simulated a = {find_g_a(C_g, C_B, f_r, Lj, 4)[1]} Linear MHz")
print(f"Simulated f_q = {find_g_a(C_g, C_B, f_r, Lj, 4)[2]} Linear GHz")

Qubit 1
Experimental g = 61 Linear MHz
Simulated g = 67.30512966260582 Linear MHz
Simulated a = -164.61390750785299 Linear MHz
Simulated f_q = 4.283443913871817 Linear GHz


### Load LOM json (worth checking if the filename matches the qubit index)

In [6]:
lom_json = open(f"qubitonly_num{index}_v4.json")

data_lom = json.load(lom_json)

print(data_lom["data"])

{'cross_to_ground': -87.42696, 'claw_to_ground': -79.9118, 'cross_to_claw': -5.99077, 'cross_to_cross': -87.42696, 'claw_to_claw': 86.33638, 'ground_to_ground': 216.15799}


### Load Eigenmode json (unfortunately you're gonna have to paste in the custom filename here)

In [7]:
import json

In [8]:
emode_json = open(f"cavity_claw_num{index}_v4.json") # PASTE IN CUSTOM FILENAME

data_emode = json.load(emode_json)

# print(data_emode["design_options"]["geometry_dict"]["claw_opts"]["hfss_inductance"][0])
print(data_emode["sim_results"])

{'cavity_frequency': 6370362676.89, 'Q': 10880057.294178646, 'kappa': 585.508192158}


## Generate data (np.dataframe) using above method

In [9]:
data = {}

ind = 1 ### UPDATE INDEX (this is for bookkeeping purposes only though)

cross2cpw = abs(data_lom["data"]["cross_to_claw"]) * 1e-15
cross2ground = abs(data_lom["data"]["cross_to_ground"]) * 1e-15
f_r = data_emode["sim_results"]["cavity_frequency"]
Lj = data_lom["design_options"]["hfss_inductance"] * 1e-9 #if data_emode["design_options"]["geometry_dict"]["claw_opts"]["hfss_inductance"][0] == data_lom["design_options"]["hfss_inductance"] else -1
gg, aa, ff_q = find_g_a(cross2cpw, cross2ground, f_r, Lj, N=4)
data.update({f"qubit_{ind}": dict(
    f_cavity = f_r,
    Q = data_emode["sim_results"]["Q"],
    kappa = data_emode["sim_results"]["kappa"],
    g = gg,
    a = aa,
    f_qubit = ff_q
)})

print(data)


{'qubit_1': {'f_cavity': 6370362676.89, 'Q': 10880057.294178646, 'kappa': 585.508192158, 'g': 83.25449261270606, 'a': -235.42659962462054, 'f_qubit': 4.225014693531719}}


In [10]:
data

{'qubit_1': {'f_cavity': 6370362676.89,
  'Q': 10880057.294178646,
  'kappa': 585.508192158,
  'g': 83.25449261270606,
  'a': -235.42659962462054,
  'f_qubit': 4.225014693531719}}

### Export to json (if desired)

In [11]:
from utils import *
save_simulation_data_to_json(data, f"final_q{index}_v4")